In [1]:
from __future__ import division, print_function
import pandas as pd
import glob as glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import optimize
%matplotlib inline
sns.set_style('white')

### Set parameters

In [ ]:
N_samples = 10000
threshold = .5 
REWARD = 1

### Free parameters to fit

In [ ]:
#COST = 0.5
#B=6
#A=60 
#u_skill = .625 #mean of beta
#tao_skill = 20 #variance of beta
#skill=.4
#scale = 200

### Functions

In [8]:
def prob_reward(e,diff,skill, A, B):
    """Return the probability of getting the reward
    
    Parameters
    ----------
    e : numeric or array-like
        Description of this parameter.
    # TODO: list other parameters
    
    Returns
    -------
    ...
    """
    return 1/(1 + A * np.exp(-(e * skill / diff) * B)) 

def get_effort(cost, diff, skill, A, B,reward=1):
    """Documentation."""
    es = np.linspace(0, 1, 100) 
    utilities = (
        reward * prob_reward(e=es, diff=diff, skill=skill, A=A, B=B)
        - cost * es
    )
    return es[np.argmax(utilities)]  #return argmax of utility

def your_eff_know_skill(n_samples, cost, diff, skill, A, B,reward=1):  #model for planning. Takes in diff, skill, and cost -returns effort
    """Documentation."""
    sampled_effort = np.empty(n_samples)
    for i in xrange(n_samples):
        d = np.random.choice(diff)
        effort=get_effort(cost=cost, diff=d, skill=skill, A=A, B=B, reward=reward)
        sampled_effort[i] = effort
    
    return sampled_effort


def create_priors(n_samples, tao_skill, u_skill, cost, A, B, reward=1,threshold=0.5):
    """Documentation."""
    # Skill and diff are beta distributions.
    skills = np.random.beta(
        tao_skill * u_skill, (1 - u_skill) * tao_skill, size=n_samples,
    )
    diffs = np.random.beta(.5,.5, size=n_samples)

    # Create array of effort values, for each value of skill and diff.
    # This initializes an empty array and fills in a value with each 
    # iteration.
    # TODO: this seems to be the bottleneck. Is there any way this can
    #       be vectorized?
    efforts = np.empty_like(skills)
    for ii, (this_skill, this_diff) in enumerate(zip(skills, diffs)):
        efforts[ii] = get_effort(
            cost=cost, skill=this_skill, diff=this_diff, A=A, B=B
        )

    high_effort = efforts > threshold
    success = prob_reward(efforts, diffs, skills, A, B) > np.random.random(n_samples)

    return skills, diffs, success, high_effort

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Function that takes in free parameters and spits out model predictions

In [47]:
def run_model(cost, A, B, u_skill,tao_skill, skill, scale,n_samples=1000):
    REWARD=1
    skills, diffs, success, high_effort= create_priors(
        n_samples =n_samples, tao_skill=tao_skill, u_skill=u_skill, cost=cost, A=A, B=B
    )

    effort_high_success = your_eff_know_skill(n_samples, cost, diffs[success & high_effort],skill, A,B) 
    effort_low_success = your_eff_know_skill(n_samples,  cost, diffs[success & (~high_effort)],skill,A,B) 
    effort_high_fail = your_eff_know_skill(n_samples,  cost, diffs[(~success) & high_effort],skill,A,B) 
    effort_low_fail = your_eff_know_skill(n_samples,  cost, diffs[(~success) & (~high_effort)],skill,A,B) 
    effort_baseline = your_eff_know_skill(n_samples,  cost, np.random.beta(.5, .5, size=n_samples),skill,A,B)

    index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"]
    d = {
        'Model' : pd.Series(
            [ 
                np.mean(effort_high_success) * scale, 
                np.mean(effort_low_success) * scale, 
                np.mean(effort_high_fail) * scale, 
                np.mean(effort_low_fail) * scale, 
                np.mean(effort_baseline)*scale
            ], index=index),
         'Human' : pd.Series(
            [107, 57, 30, 35,86], 
            index=index)}
    
    df = pd.DataFrame(d)
    rmse_val = rmse( df['Model'],df['Human'])
    return rmse_val

In [48]:
def wrapped_run_model(x):
    return run_model(*x)

In [54]:
inputs = np.array([0.5,60,6, .625, 20, .4, 200])
%timeit wrapped_run_model(inputs)
output

1 loop, best of 3: 226 ms per loop


18.94004073777716

### Optminize function

In [56]:
key_words=dict(cost = 0.5,B=6,A=60 ,u_skill = .625 ,tao_skill = 20, skill=.4, scale =200)
rmse_val= run_model(**key_words)

In [63]:
inputs = np.array([0.5,60,6, .625, 20, .4, 200])
minimum = optimize.fmin(wrapped_run_model, inputs, maxiter =1000)

In [64]:
minimum

array([   0.45503125,   61.4582183 ,    6.13676878,    0.63085187,
         20.34939935,    0.40989297,  204.30968831])

### Testing

In [40]:
cost = 0.5
B=6
A=60 
u_skill = .625 #mean of beta
tao_skill = 20 #variance of beta
skill=.4
scale = 200
threshold = .5
skills, diffs, success, high_effort=create_priors(1000, tao_skill, u_skill, cost, A, B)

In [41]:
efforthigh_success = your_eff_know_skill(1000,cost, diffs[success & high_effort],skill, A,B) 
effortlow_success = your_eff_know_skill(1000,cost, diffs[success & (~high_effort)],skill,A,B) 
efforthigh_fail = your_eff_know_skill(1000,cost, diffs[(~success) & high_effort],skill,A,B) 
effortlow_fail = your_eff_know_skill(1000,cost, diffs[(~success) & (~high_effort)],skill,A,B) 
effortbaseline = your_eff_know_skill(1000,cost, np.random.beta(.5, .5, size=100),skill,A,B)

In [11]:
scale = 200
d = {'Model' : pd.Series([np.mean(efforthigh_success)*scale, np.mean(effortlow_success)*scale, np.mean(efforthigh_fail)*scale, np.mean(effortlow_fail)*scale, np.mean(effortbaseline)*scale], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"]),
     'Human' : pd.Series([107, 57, 30, 35,85], index=['High Effort Success', 'Low Effort Success', 'High Effort Failure', 'Low Effort Failure',"Baseline"])}

df = pd.DataFrame(d)
df

,Human,Model
High Effort Success,107,94.309091
Low Effort Success,57,55.747475
High Effort Failure,30,29.325253
Low Effort Failure,35,6.096970
Baseline,85,66.597980
